<a href="https://www.kaggle.com/code/shahrish99/cornet-s-pgd-with-mnist?scriptVersionId=256937293" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cornet-s_mnist_redo/pytorch/default/1/cornet_s-mnist-redo.pth
/kaggle/input/cornet-s/pytorch/default/1/cornet_s-1d3f7974.pth
/kaggle/input/cornet-s-mnist/pytorch/default/1/cornet_s_MNIST.pth


## Loading the dataset with labels

In [3]:
!pip install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchattacks

In [4]:
# Define transformations for MNIST
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),  # Resize for CORNet-S input
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize MNIST
])

In [5]:
# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 38308247.58it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 1169094.18it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 10711119.54it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 3448059.51it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
!git clone https://github.com/dicarlolab/CORnet.git

Cloning into 'CORnet'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 155 (delta 13), reused 9 (delta 8), pack-reused 135 (from 1)
Receiving objects: 100% (155/155), 68.11 KiB | 1.55 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [8]:
# Navigate to the cloned repository folder
import os
os.chdir('/kaggle/working/CORnet')

# Install the package if needed
!pip install .

Processing /kaggle/working/CORnet
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for CORnet: filename=CORnet-0.1.0-py3-none-any.whl size=23228 sha256=30a5b76f7ec24bbfba3c6d8fcd3d2c51dfa27c8445a9a5e3e6e7837db0d33361
  Stored in directory: /tmp/pip-ephem-wheel-cache-dg40u8e3/wheels/ab/bb/f9/8716bf8cc3f23c0cd07d33b31c64ed0bc87023663d6be90ad5
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=99da034ed40926b63bd55b320744e336f598453a1d63bc753135e323603d0f76
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built CORnet fire


In [9]:
# Import the CORnet-S model architecture
from cornet import cornet_s

# Initialize the CORnet-S model
model = cornet_s()

print(model) # Print the model

DataParallel(
  (module): Sequential(
    (V1): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nonlin2): ReLU(inplace=True)
      (output): Identity()
    )
    (V2): CORblock_S(
      (conv_input): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (skip): Conv2d(128, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (norm_skip): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (no

In [ ]:
import torch.nn as nn

# Load pre-trained weights without modifying the final layer
checkpoint = torch.load("/kaggle/input/cornet-s/pytorch/default/1/cornet_s-1d3f7974.pth", map_location=device)
model.load_state_dict(checkpoint['state_dict'], strict=False)  # Use strict=False to ignore final layer mismatch

## Training procedure

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Import necessary modules
import time
from tqdm import tqdm

# Move model to the appropriate device
model = model.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0

    # Training phase
    for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item() * inputs.size(0)  # Update running loss

        if i % 100 == 0:  # Print every 100 batches
            print(f"Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader.dataset)  # Compute epoch loss

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct1 = 0
    correct5 = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss

            val_loss += loss.item() * inputs.size(0)  # Update validation loss

            # Calculate top-1 and top-5 accuracy
            _, pred1 = outputs.topk(1, 1, True, True)
            _, pred5 = outputs.topk(5, 1, True, True)
            correct1 += pred1.eq(labels.view(-1, 1).expand_as(pred1)).sum().item()
            correct5 += pred5.eq(labels.view(-1, 1).expand_as(pred5)).sum().item()
            total += labels.size(0)

    epoch_val_loss = val_loss / len(test_loader.dataset)  # Compute epoch validation loss
    top1_acc = correct1 / total  # Compute top-1 accuracy
    top5_acc = correct5 / total  # Compute top-5 accuracy

    end_time = time.time()
    epoch_duration = end_time - start_time

    print(f'Epoch [{epoch+1}/{num_epochs}] - Time: {epoch_duration:.2f}s')
    print(f'Train Loss: {epoch_loss:.4f}')
    print(f'Validation Loss: {epoch_val_loss:.4f}')
    print(f'Top-1 Accuracy: {top1_acc:.4f}, Top-5 Accuracy: {top5_acc:.4f}')

    # Save the model checkpoint
    model_save_path = f'/kaggle/working/cornet_s_epoch{epoch+1}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved at {model_save_path}')

print('Training complete')

In [ ]:
# Load the model checkpoint
model_path = '/kaggle/working/cornet_s_epoch1.pth'
model.load_state_dict(torch.load(model_path))

In [ ]:
# Testing model out with some images:
import matplotlib.pyplot as plt
import torch

def show_images(images, labels, preds=None, num_images=6):
    """
    Function to display a grid of images with corresponding labels and predictions.
    """
    images = images[:num_images]
    labels = labels[:num_images]

    # Create a grid of images
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i in range(num_images):
        # Convert the tensor to a NumPy array for displaying
        img = images[i].cpu().numpy().transpose((1, 2, 0))
        axes[i].imshow(img, cmap='gray' if img.shape[2] == 1 else None)
        axes[i].axis('off')

        # Display label and prediction
        title = f"Label: {labels[i].item()}"
        if preds is not None:
            title += f"\nPred: {preds[i].item()}"
        axes[i].set_title(title)

    plt.show()

# Example usage after training the model on clean images:
model.eval()
data_iter = iter(test_loader)
images, labels = next(data_iter)

# Run the model on the clean images
outputs = model(images.to(device))
_, preds = torch.max(outputs, 1)

# Visualize the clean images and predictions
show_images(images, labels, preds)

## PGD Attacks

In [11]:
import torch.nn as nn

# Load pre-trained weights without modifying the final layer
checkpoint = torch.load("/kaggle/input/cornet-s-mnist/pytorch/default/1/cornet_s_MNIST.pth", map_location=device)

model.load_state_dict(checkpoint)

/tmp/ipykernel_36/2349068480.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/input/cornet-s-mnist/pytorch/default/1/cornet_s_MNIST.pth",

<All keys matched successfully>

In [1]:
# PGD Attack
pgd_attack = torchattacks.PGD(model, eps=0.001, alpha=2/255, steps=40)

# Function to test adversarial accuracy
def adversarial_test(attack, loader, attack_name):
    model.eval()
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(loader, desc=f"Adversarial Test ({attack_name})"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        
        _, pred = outputs.max(1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)
    
    acc = correct / total
    print(f'{attack_name} Accuracy: {acc:.4f}')
    return acc

# Evaluate model with PGD attack
pgd_acc = adversarial_test(pgd_attack, test_loader, "PGD")

NameError: name 'torchattacks' is not defined

In [ ]:
# PGD Attack for Adversarial Training
pgd_attack_train = torchattacks.PGD(model, eps=0.3, alpha=2/255, steps=40)

# Function for Adversarial Training with PGD
def adversarial_train_pgd(loader, model, optimizer, criterion, attack):
    model.train()
    progress_bar = tqdm(enumerate(loader), total=len(loader), desc="Training")
    
    for i, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate PGD adversarial examples
        adv_inputs = attack(inputs, labels)
        
        # Forward pass with adversarial inputs
        optimizer.zero_grad()
        outputs = model(adv_inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update progress bar with loss
        progress_bar.set_postfix(loss=loss.item())

# Training loop (e.g., 10 epochs)
for epoch in range(10):
    adversarial_train_pgd(train_loader, model, optimizer, criterion, pgd_attack_train)
    print(f"Epoch {epoch+1} completed.")

In [ ]:
# Re-test model against clean data and PGD attack

# Test on clean data
clean_accuracy = adversarial_test(None, test_loader, "Clean")
print(f'Accuracy on clean data after PGD adversarial training: {clean_accuracy * 100:.2f}%')

# Test on PGD adversarial examples
pgd_acc_after = adversarial_test(pgd_attack, test_loader, "PGD")
print(f'Accuracy under PGD Attack after adversarial training: {pgd_acc_after * 100:.2f}%')

# Adversarial FGSM Training and Attacks
* This section will conduct FGSM attacks, and then we will make the model more robust by training it with adversarial attacks, and then retest it with FGSM attacks.
* Issue with this is that FGSM attacks are not effective against this model, it still has a 95% accuracy. We will go back to PGD attacks then. 

## FGSM Attacks
* We first need to see the accuracy that CORnet-S can achieve with FGSM attacks.


In [ ]:
import torchattacks
from tqdm import tqdm

# Define FGSM attack
fgsm = torchattacks.FGSM(model, eps=0.3)

# Function to evaluate under FGSM attack with TQDM
def evaluate_under_attack(loader, model, attack):
    model.eval()
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(loader), total=len(loader), desc="Evaluating")
    for i, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate adversarial examples if attack is specified
        if attack:
            adv_inputs = attack(inputs, labels)
        else:
            adv_inputs = inputs
        
        outputs = model(adv_inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Update the progress bar with current accuracy
        progress_bar.set_postfix(batch_accuracy=(correct / total))
    
    return correct / total

# Test the model under FGSM attack
accuracy_fgsm = evaluate_under_attack(test_loader, model, fgsm)
print(f'Accuracy under FGSM Attack: {accuracy_fgsm * 100:.2f}%')

### Adversarial Training

In [ ]:
# Function for adversarial training with progress bar
def adversarial_train(loader, model, optimizer, criterion, attack):
    model.train()
    progress_bar = tqdm(enumerate(loader), total=len(loader), desc="Training")
    for i, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate adversarial examples using FGSM
        adv_inputs = attack(inputs, labels)
        
        # Forward pass with adversarial inputs
        optimizer.zero_grad()
        outputs = model(adv_inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update progress bar with loss
        progress_bar.set_postfix(loss=loss.item())

# Adversarial training loop (e.g., 10 epochs)
for epoch in range(10):
    adversarial_train(train_loader, model, optimizer, criterion, fgsm)
    print(f"Epoch {epoch+1} completed.")

### Re-Test

In [ ]:
# Evaluate on clean data after adversarial training
clean_accuracy = evaluate_under_attack(test_loader, model, None)
print(f'Accuracy on clean data after FGSM adversarial training: {clean_accuracy * 100:.2f}%')

# Evaluate on FGSM attack again after adversarial training
accuracy_fgsm_after = evaluate_under_attack(test_loader, model, fgsm)
print(f'Accuracy under FGSM Attack after adversarial training: {accuracy_fgsm_after * 100:.2f}%')

## CW Attacks and Training
* Since the model is scoring so well against FGSM and PGD attacks, we can use CW attacks to find a bad accuracy and then improve it with adversarial training.

In [17]:
import torchattacks
from tqdm import tqdm

# CW Attack Setup
cw_attack = torchattacks.CW(model, c=1e-3, kappa=0, steps=100, lr=0.01)

# Function to test model accuracy on CW attack with TQDM
def adversarial_test(attack, loader, attack_name):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    # Iterate through the data loader with a progress bar
    for inputs, labels in tqdm(loader, desc=f"Adversarial Test ({attack_name})"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate adversarial examples
        adv_inputs = attack(inputs, labels)
        
        # Get model outputs from adversarial inputs
        outputs = model(adv_inputs)
        _, pred = outputs.max(1)  # Get predicted class
        
        correct += pred.eq(labels).sum().item()  # Count correct predictions
        total += labels.size(0)  # Total number of labels
    
    # Calculate accuracy
    acc = correct / total
    print(f'{attack_name} Accuracy: {acc:.4f}')
    return acc

# Test model accuracy on CW attack
cw_acc = adversarial_test(cw_attack, test_loader, "CW")
print(f'Accuracy under CW Attack: {cw_acc * 100:.2f}%')

Adversarial Test (CW): 100%|██████████| 157/157 [25:14<00:00,  9.65s/it]

CW Accuracy: 0.9712
Accuracy under CW Attack: 97.12%


In [ ]:
# CW Adversarial Training Function
def adversarial_train_cw(loader, model, optimizer, criterion, attack):
    model.train()
    progress_bar = tqdm(enumerate(loader), total=len(loader), desc="CW Adversarial Training")
    
    for i, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Generate CW adversarial examples
        adv_inputs = attack(inputs, labels)
        
        # Forward pass with adversarial inputs
        optimizer.zero_grad()
        outputs = model(adv_inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update progress bar with loss
        progress_bar.set_postfix(loss=loss.item())

# Train model on CW adversarial examples
for epoch in range(10):  # Adjust number of epochs based on needs
    adversarial_train_cw(train_loader, model, optimizer, criterion, cw_attack)
    print(f"Epoch {epoch+1} completed.")

In [ ]:
# Re-Test model on CW after training
cw_acc_after = adversarial_test(cw_attack, test_loader, "CW After Training")
print(f'Accuracy under CW Attack after adversarial training: {cw_acc_after * 100:.2f}%')

# Show CW adversarial examples again after training
show_adversarial_examples(test_loader, cw_attack, num_images=5)


# Redo

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

!git clone https://github.com/dicarlolab/CORnet.git

# Navigate to the cloned repository folder
import os
os.chdir('/kaggle/working/CORnet')

# Install the package if needed
!pip install .

# Import the CORnet-S model architecture
from cornet import cornet_s
# CORnet-S model
model = cornet_s(pretrained=False).to(device)
model.module.decoder.linear = nn.Linear(model.module.decoder.linear.in_features, 10).to(device)
print(model)

cuda
Cloning into 'CORnet'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 155 (delta 13), reused 9 (delta 8), pack-reused 135 (from 1)
Receiving objects: 100% (155/155), 68.11 KiB | 2.62 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Processing /kaggle/working/CORnet
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for CORnet: filename=CORnet-0.1.0-py3-none-any.whl size=23228 sha256=a3a05822d318f21d0b4702dd4db1c758b8520a6b8d545653d52210c47e25b38e
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5p8bb3q/wheels/ab/bb/f9/8716bf8cc3f23c0cd07d33b31c64ed0bc87023663d6be90ad5
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=ef76b328cbe6bc8dab7b7f7dfc50aaed152c098567054fa63c91be867f74b381
  Stored in directory: /root/.cache/pi

In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

# --- Transforms for MNIST (resize + convert to 3-channel for CORNet-S) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),              # CORNet-S expects larger images
    transforms.Grayscale(num_output_channels=3),# Convert 1-channel MNIST to 3
    transforms.ToTensor()
])

# --- Datasets ---
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# --- Loss and Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Training Function ---
def train(model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    pbar = tqdm(train_loader, desc=f"Train Epoch {epoch}")
    total_loss, correct, total = 0, 0, 0

    for data, target in pbar:
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

        pbar.set_postfix(loss=total_loss/total, acc=correct/total)

# --- Testing Function ---
def test(model, device, test_loader, criterion):
    model.eval()
    pbar = tqdm(test_loader, desc="Testing")
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

            pbar.set_postfix(loss=total_loss/total, acc=correct/total)

    return total_loss / total, correct / total

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 34414216.11it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 1042987.47it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 9427101.57it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 2883385.62it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch, criterion)
    test(model, device, test_loader, criterion)

Train Epoch 1:   0%|          | 0/938 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Train Epoch 5:   1%|          | 5/938 [00:05<18:39,  1.20s/it, acc=0.991, loss=0.0281]


KeyboardInterrupt: 

In [3]:
!pip install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.

In [4]:
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
torch.save(model.state_dict(), "/kaggle/working/cornet_s-mnist-redo.pth")

In [5]:
import torch.nn as nn

# Load pre-trained weights without modifying the final layer
checkpoint = torch.load("/kaggle/input/cornet-s_mnist_redo/pytorch/default/1/cornet_s-mnist-redo.pth", map_location=device)

model.load_state_dict(checkpoint)

/tmp/ipykernel_36/3750294667.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/input/cornet-s_mnist_redo/pytorch/default/1/cornet_s-mnist-

<All keys matched successfully>

In [ ]:
import torchattacks
from tqdm import tqdm


# --- PGD Attack Configuration ---
pgd_eps = 0.001     
pgd_alpha = 2/255
pgd_steps = 40

pgd_attack = torchattacks.PGD(model, eps=pgd_eps, alpha=pgd_alpha, steps=pgd_steps)

# --- PGD Adversarial Accuracy Function ---
def adversarial_test_pgd(attack, loader):
    model.eval()
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc="Adversarial Test (PGD)"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"📊 PGD Accuracy (ε={pgd_eps}): {acc:.4f}")
    return acc

# --- Run PGD Evaluation ---
pgd_acc = adversarial_test_pgd(pgd_attack, test_loader)

Adversarial Test (PGD):   0%|          | 0/157 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Adversarial Test (PGD):  32%|███▏      | 50/157 [19:05<41:55, 23.51s/it]

In [8]:
test(model, device, test_loader, criterion)

Testing: 100%|██████████| 313/313 [00:47<00:00,  6.62it/s, acc=0.99, loss=0.0306] 


(0.030556247400678693, 0.9899)

In [ ]:
import torchattacks
from tqdm import tqdm

# ---------------- CW Attack Setup ----------------
cw_c = 1e-1          # L2 perturbation penalty
cw_kappa = 0         # Confidence
cw_steps = 100
cw_lr = 0.01

cw_attack = torchattacks.CW(model, c=cw_c, kappa=cw_kappa, steps=cw_steps, lr=cw_lr)

# ---------------- CW Adversarial Accuracy Function ----------------
def adversarial_test_cw(attack, loader):
    model.eval()
    correct = 0
    total = 0

    for inputs, labels in tqdm(loader, desc=f"Adversarial Test (CW)"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"📊 CW Accuracy (c={cw_c}, kappa={cw_kappa}): {acc:.4f}")
    return acc

# ---------------- Run CW Evaluation ----------------
cw_acc = adversarial_test_cw(cw_attack, test_loader)

Adversarial Test (CW):  79%|███████▉  | 248/313 [23:26<06:13,  5.74s/it]